<a href="https://colab.research.google.com/github/AYDOSUL/Pings/blob/main/VQELearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pennylane --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 94.8 MB/s eta 0:00:00


In [ ]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem

symbols = ["H", "O", "H"]
coordinates = np.array([[-0.0399, -0.0038, 0.0], [1.5780, 0.8540, 0.0], [2.7909, -0.5159, 0.0]])

molecule = qchem.Molecule(symbols, coordinates)
H, qubits = qchem.molecular_hamiltonian(molecule)

# Create quantum device
dev = qml.device("default.qubit", wires=qubits)

# Define QNode to compute energy
@qml.qnode(dev)
def exp_energy(state):
    qml.BasisState(np.array(state), wires=range(num_wires))
    return qml.expval(H)
print(qubits)
# Generate Hartree-Fock state for H2O (This uses the correct number of qubits and the Hartree-Fock occupation)
hf = qchem.hf_state(electrons=10, orbitals=qubits)

# Ensure that hf has the right length
print("HF state:", hf)

# Print HF energy
print("HF Energy:", exp_energy(hf))

14
HF state: [1 1 1 1 1 1 1 1 1 1 0 0 0 0]
HF Energy: -74.96452572011275


In [ ]:
#HF No VQE with H2
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem

symbols = ["H", "H"]
coordinates = np.array([[-0.673,0,0],[0.673,0,0]])

hamiltonian, qubits = qchem.molecular_hamiltonian(symbols, coordinates)
num_wires = qubits
dev = qml.device("default.qubit", wires=num_wires)
print(qubits)
@qml.qnode(dev)
def exp_energy(state):
  qml.BasisState(np.array(state), wires=range(num_wires))
  return qml.expval(hamiltonian)

hf = qchem.hf_state(electrons=2, orbitals=4)
print(exp_energy(hf)) #Lowest Energy State

4
-1.1175058833436393


In [ ]:
#H2O VQE
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem

# Define the molecule
symbols = ["H", "O", "H"]
coordinates = np.array([[-0.0399, -0.0038, 0.0], [1.5780, 0.8540, 0.0], [2.7909, -0.5159, 0.0]])

# Generate the molecular Hamiltonian
molecule = qchem.Molecule(symbols, coordinates)
H, qubits = qchem.molecular_hamiltonian(molecule)

# Number of qubits (the same as the number of orbitals used)
num_wires = qubits

# Hartree-Fock state for 10 electrons and 10 orbitals
hf = qchem.hf_state(electrons=10, orbitals=num_wires)

# VQE Setup
dev = qml.device("default.qubit", wires=num_wires)

# Energy measurement function
def exp_energy(state):
    qml.BasisState(np.array(state), wires=range(num_wires))
    return qml.expval(H)

# Ansatz function
def ansatz(params):
    qml.BasisState(hf, wires=range(num_wires))  # Start with the Hartree-Fock state
    # Apply double-excitation gates with valid qubit indices
    qml.DoubleExcitation(params[0], wires=[0, 1, 2, 3])  # DoubleExcitation on qubits 0, 1, 2, 3
    qml.DoubleExcitation(params[1], wires=[4, 5, 6, 7])  # DoubleExcitation on qubits 4, 5, 6, 7

@qml.qnode(dev)
def cost_function(params):
    ansatz(params)
    return qml.expval(H)

# Optimizer
opt = qml.GradientDescentOptimizer(stepsize=0.1)
theta = np.array([0.0, 0.0], requires_grad=True)

# Energy and angle tracking
energy = [cost_function(theta)]
angle = [theta]
max_iterations = 2

# Optimization loop
for n in range(max_iterations + 1):
    theta, prev_energy = opt.step_and_cost(cost_function, theta)

    energy.append(cost_function(theta))
    angle.append(theta)

    print(f"Step = {n}, Energy = {energy[-1]:.8f} Ha")  # Log Prints

# Function to compute the final quantum state
@qml.qnode(dev)
def ground_state(params):
    ansatz(params)
    return qml.state()

# Print the ground state of the system
print(ground_state(theta))


Exception ignored in: <function _xla_gc_callback at 0x78799085b9c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 
Exception ignored in: <function _xla_gc_callback at 0x78799085b9c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 
Exception ignored in: <function _xla_gc_callback at 0x78799085b9c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
#H3 VQE
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem

symbols = ["H", "H", "H"]
coordinates = np.array([[0.0102,0.0442,0.0],[0.9867,1.6303,0.0],
                        [1.8720,-0.0085,0.0]])#Vertices of H_3 atom
hamiltonian, qubits = qchem.molecular_hamiltonian(symbols, coordinates, charge=1)
print(qubits)
hf = qchem.hf_state(electrons=2, orbitals = 6)

num_wires = qubits
dev = qml.device("default.qubit", wires=num_wires)

@qml.qnode(dev)
#Calculates Energy
def exp_energy(state):
  qml.BasisState(np.array(state), wires=range(num_wires))
  return qml.expval(hamiltonian)
#Ansatz
def ansatz(params):
  qml.BasisState(hf,wires=range(num_wires))
  qml.DoubleExcitation(params[0],wires=[0,1,2,3])
  qml.DoubleExcitation(params[1],wires=[0,1,4,5])
#Cost function given parameters
@qml.qnode(dev)
def cost_function(params):
  ansatz(params)
  return qml.expval(hamiltonian)

#Optimizer
opt = qml.GradientDescentOptimizer(stepsize=0.1)
theta = np.array([0.0,0.0], requires_grad = True)

energy = [cost_function(theta)]
angle = [theta]
max_iterations = 50

for n in range(max_iterations + 1):
  theta, prev_energy = opt.step_and_cost(cost_function, theta)

  energy.append(cost_function(theta))
  angle.append(theta)

  print(f"Step = {n}, Energy = {energy[-1]:.8f} Ha")#Log Prints

@qml.qnode(dev)
def ground_state(params):
  ansatz(params)
  return qml.state()
print(ground_state(theta))

6
Step = 0, Energy = -1.25054570 Ha
Step = 1, Energy = -1.25397718 Ha
Step = 2, Energy = -1.25692185 Ha
Step = 3, Energy = -1.25944706 Ha
Step = 4, Energy = -1.26161130 Ha


KeyboardInterrupt: 